# RAG(Retrieval-Augmented Generation) based QA Chat bot

## 0. 미션
참조
- 정보: https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma?hl=ko
- 2b: https://huggingface.co/google/gemma-2b
- 2b instruction tuning: https://huggingface.co/google/gemma-1.1-2b-it
- 7b: https://huggingface.co/google/gemma-7b
- 7b instruction tuning: https://huggingface.co/google/gemma-1.1-7b-it
- BM25: https://github.com/dorianbrown/rank_bm25
- SentenceTransformers: https://www.sbert.net/

미션
- 질문에 대해서 적절한 문서를 검색하고, 검색된 문서에 근거해서 답변하는 RAG 챕봇을 만들어봅니다.
- 문서 검색은 2일차 실습 결과를 사용합니다.
- 검색된 문서에 근거해서 답변하는 기능은 구글의 gemma-2b-it SLLM을 사용합니다.
- 필요에 따라서 gemma-2b-it를 fine-tuning 합니다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [ ]:
!pip install -q -U transformers==4.38.2
!pip install -q -U datasets==2.18.0
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.9.0
!pip install -q -U trl==0.7.11
!pip install -q -U accelerate==0.27.2
!pip install -q -U rank_bm25==0.2.2
!pip install -q -U sentence-transformers==2.7.0
!pip install -q -U wikiextractor==3.0.6
!pip install -q -U konlpy==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cup

## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## *3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [ ]:
import os
import glob
import json

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import konlpy
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

### 3.2. HuggingFace login
- 이번 프로젝트는 HuggingFace 로그인 해야만 진행이 가능합니다.
- HuggingFace 계정이 없다면 아래 URL에 접속해서 가입하시기 바랍니다.
  - https://huggingface.co/
- HuggingFace 로그인을 위해서 아래 URL에 접속해서 'User Access Token'을 생성하고 복사해서 Token에 입력하세요.
  - https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# access token을 복사하세요.
HF_TOKEN = ""

### 3.3. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- SEARCH_MODEL_ID
  - 검색을 위한 SentenceTransformer 입니다.
  - 서울대학교 컴퓨터언어학_자연어처리 연구실에서 공개한 모델입니다.
  - https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
- SLLM_MODEL_ID
  - 문서에 근거해서 답변 기능을 위한 SLLM 입니다.
  - 구글에서 공개한 gemma-2b를 Instruction tunned한 버전입니다.
  - https://huggingface.co/google/gemma-2b-it
- CHUNK_FN
  - 문서를 일정한 단위로 분할해서 저장할 파일 이름
  - 데이터베이스를 대신하는 역할
  - 실제 기능을 만들 때는 DB를 사용해야 합니다.


In [ ]:
WORKSPACE = '/content/drive/MyDrive/nlp-project'
SEARCH_MODEL_ID = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
SLLM_MODEL_ID = 'google/gemma-1.1-2b-it'
SLLM_MODEL_ID2 = 'google/gemma-1.1-7b-it'
CHUNK_FN = os.path.join(WORKSPACE, "data", "chunk_db.json")

## 4. SLLM RAG tutorial (재시작 필요)
- SLLM에 질문과 근거 문서를 함께 입력하고 질문에 맞는 답변을 근거 문서로 부터 하도록 하는 과정을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 4.1. model load with 4 bits
- 2B token을 가진 gemma를 그냥 로딩할 경우는 약 9G의 GPU vRAM이 필요합니다.
- 4bit 양자화를 할 경우 2.2G의 GPU vRAM 필요.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

### 4.2. pipeline
- https://huggingface.co/docs/transformers/main_classes/pipelines
- huggingface에서 inference를 쉽게 하기 위해 정의한 라이브러리.

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

### 4.3. sllm prompt
- https://huggingface.co/google/gemma-1.1-2b-it
- 아래와 같은 형식이 gemma의 promt 형식 입니다.
```
<bos><start_of_turn>user
{content}<end_of_turn>
<start_of_turn>model
```
- NSMC 추론을 위한 프롬프트를 생성하는 과정입니다.

In [ ]:
query = "지미 카터 대통령이 졸업한 대학교는?"
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {chunk_list[4]}

문서4: {chunk_list[3]}

문서3: {chunk_list[2]}

문서2: {chunk_list[1]}

문서1: {chunk_list[0]}

질문: {query}"""
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
print(prompt)

### 4.4. sllm inference
- 이전 단계에서 생성한 prompt를 이용해 추론하고 결과를 확인하는 과장입니다.

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
outputs

In [ ]:
print(outputs[0]["generated_text"])

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

### 4.5. sllm chatbot
- chatbot 형식의 QA 예 입니다.

In [ ]:
# 프롬프트 생성 함수
def gen_prompt(pipe, chunk_list, query):
    messages = [
        {
            "role": "user",
            "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {chunk_list[4]}

문서4: {chunk_list[3]}

문서3: {chunk_list[2]}

문서2: {chunk_list[1]}

문서1: {chunk_list[0]}

질문: {query}"""
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = gen_response(pipe, chunk_list, query)
    print(f'답변 > {result}\n\n')

## 5. SLLM no RAG 실습 (재시작 필요)
- RAG를 사용하지 않고 입력한 질문만을 이용해서 SLLM의 답변 능력을 확인하는 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 5.1. SLLM no RAG
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문을 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

## 6. SLLM RAG with BM25 실습 (재시작 필요)
- BM25의 검색 기능과 SLLM의 추론 기능을 합쳐 RAG를 구성하는 실습입니다.
- BM25에서 tokenizer를 사용합니다.
- SLLM에서도 tokenizer를 사용합니다.
- 두 tokeinzer에 각각 다른 구분할 수 있는 이름을 변수 명으로 선언해야 합니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 6.1. bm25를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.
- BM25를 위해서 tokenizer를 정의하고 전처리를 실행합니다.

#### 전처리된 파일 불러오기

In [ ]:
import gdown, os, glob, shutil
import csv

def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

# 파일 다운로드
file_id = '1wrlIHXvZXLBoiPjH2nIPURdceMusQdrj'
download_file(file_id, 'chunk_list.csv')
file_id = '1fBk2ohMp1GJbGrSi8P0-zcIugW-csHUi'
download_file(file_id, "tokenized_chunks.csv")


#  문서를 이미 chunk 단위로 분할한 chunk_list 파일에서 불러오기
with open('/content/chunk_list.csv', "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    chunk_list = next(reader)
# chunk_list를 tokenize한 tokenized_chunks 불러오기
tokenized_chunks = []
with open('/content/tokenized_chunks.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        tokenized_chunks.append(row)

Downloading...
From (original): https://drive.google.com/uc?id=1wrlIHXvZXLBoiPjH2nIPURdceMusQdrj
From (redirected): https://drive.google.com/uc?id=1wrlIHXvZXLBoiPjH2nIPURdceMusQdrj&confirm=t&uuid=9a050aae-15b8-4e49-83cb-5715772081de
To: /content/chunk_list.csv
100%|██████████| 972M/972M [00:15<00:00, 60.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1fBk2ohMp1GJbGrSi8P0-zcIugW-csHUi
From (redirected): https://drive.google.com/uc?id=1fBk2ohMp1GJbGrSi8P0-zcIugW-csHUi&confirm=t&uuid=e29487c3-70ca-4241-94d9-5880a3c37b82
To: /content/tokenized_chunks.csv
100%|██████████| 848M/848M [00:13<00:00, 65.1MB/s]


#### tokenized_chunks에 사용된 동일한 tokenizer 정의

In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer1(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

### 6.2. BM25 검색 함수
- bm25를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
# bm25 api 생성
bm25 = BM25Okapi(tokenized_chunks)

In [ ]:

def query_bm25(bm25, query, tokens, top_n=10):
    doc_scores = bm25.get_scores(tokens)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    return rank[:top_n]

In [ ]:
# 검색어를 이용한 검색
query = "수학교과서"
tokenized_query = tokenizer1(query)
print(tokenized_query)

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

['수학', '교과서']


array([0., 0., 0., ..., 0., 0., 0.])

### 6.3. SLLM RAG with BM25 구현
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문에 대해서 관련된 문서를 bm25를 이용해서 5개 구합니다.
  - bm25의 경우 응답이 5개 이하일 수 있습니다. 이 부분을 고려하세요.
  - 질문과 함께 문서 5개를 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

#pipe line
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def bm25_query(bm25, query, tokens, chunk_list):

    result = query_bm25(bm25, query, tokens)
    print(result)
    info_list = [chunk_list[i] for i in result]
    print(info_list)
    messages = [
    {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

        문서5: {info_list[4]}

        문서4: {info_list[3]}

        문서3: {info_list[2]}

        문서2: {info_list[1]}

        문서1: {info_list[0]}

        질문: {query}"""
    }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)
    # print(prompt)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    print('답변 :', outputs[0]["generated_text"][len(prompt):])

In [ ]:
chunk_list[0]

'지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.'

In [ ]:
query = '과학이 뭐야?'
tokenized_query = tokenizer1(query)
print(tokenized_query)
# score 계산
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)
rank = np.argsort(-doc_scores)
print(rank)

['과학', '뭐', '?']
[0. 0. 0. ... 0. 0. 0.]
[920583 726610 796938 ... 338991 338978 994855]


In [ ]:
query = '과학이 뭐야'
tokens = tokenizer1(query)
query_bm25(bm25, query, tokens)

array([726610,  64143, 796938, 869527, 749616, 843375, 727336, 829939,
       965064, 920583])

In [ ]:
query = input()
tokens = tokenizer1(query)
bm25_query(bm25, query, tokens, chunk_list)

과학이 뭐야?
[920583 726610 796938 869527 829939  63426 404952 919874 883756 681867]
['종말에 뭐하세요? 바쁘세요? 구해주실 수 있나요?\n종말에 뭐하세요? 바쁘세요? 구해주실 수 있나요?(, WorldEnd)는 가레노 아키라 각본, 우에 작화의 일본의 라이트 노벨 시리즈이다. 이 시리즈의 제1권은 2014년 11월 1일 가도카와 스니커 문고를 통해 가도카와 쇼텐에 의해 출판되었다. 2016년 4월 제10권을 끝으로 시리즈는 마무리되었다. 후속작인 "종말에 뭐하세요? 다시 한번 만날 수 있나요?"(, SudaMoka)의 경우 2016년 4월 제1권부터 출판을 시작했다.', '놀면 뭐하니?\n《놀면 뭐하니?》는 매주 토요일 오후 6시 25분에 방영 중인 텔레비전 프로그램이다.', '여우 같은 게 뭐가 나빠?\n《여우 같은 게 뭐가 나빠?》()는 2020년 10월부터 TV 아사히 계열에서 방송 예정인 버라이어티 쇼이다.', 'TV야 뭐하니?\nTV야 뭐하니?는 2006년에 대교어린이TV에서 방송한 프로그램이다.\n진행자.\n이선우, 임준수\n내용.\n알루 따라잡기\nTV 소식통', '얼짱시대 요즘뭐해?\n《얼짱시대 요즘뭐해?》는 2020년부터 2021년까지 유튜브에서 방송된 웹 예능이다. 역대 얼짱시대의 얼짱들을 다시 만나 근황을 확인하는 프로그램이다.', '아바르어\n어떻게 느꼈어요?\nDur tsar chib?\n이름은 뭐에요?\nChan son due bugeb?\n몇살이에요?\nKiwe mun inev vigev?\n어디에 가세요?', '멘토\n멘토가 뭐에요?', '14대 닥터\n성격.\n14대 닥터는 10대 닥터와 거의 비슷한 것으로 보인다. 억양은 10대와 똑같이 에스추어리 영어를 쓰며, 자신의 모습이 새 모습이 아닌 이전에 겪었던 모습임을 알게 되자, 10대 닥터가 종종 외치던 "뭐야? 뭐야? 뭐야?" (What? What? What?)를 그대로 따라하는 모습을 보였다. 여기에 10대 닥터가 

## 7. SLLM RAG with SentenceTransformers 실습 (재시작 필요)
- SentenceTransformers의 검색 기능과 SLLM의 추론 기능을 합쳐 RAG를 구성하는 실습입니다.
- SentenceTransformers도 model을 사용합니다.
- SLLM도 model을 사용합니다.
- 두 model에 각각 다른 구분할 수 있는 이름을 변수 명으로 선언해야 합니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 7.1. SentenceTransformers를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.

In [ ]:
import gdown, os, glob, shutil
import csv

def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

# 파일 다운로드
file_id = '1wrlIHXvZXLBoiPjH2nIPURdceMusQdrj'
download_file(file_id, 'chunk_list.csv')

#  문서를 이미 chunk 단위로 분할한 chunk_list 파일에서 불러오기
with open('/content/chunk_list.csv', "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    chunk_list = next(reader)


In [ ]:
# SentenceTransformer 모델 생성
model_st = SentenceTransformer(SEARCH_MODEL_ID)

In [ ]:
# chunk embeddings 생성
chunk_embeddings = model_st.encode(chunk_list)
chunk_embeddings.shape

(994856, 768)

In [ ]:
type(chunk_embeddings)

numpy.ndarray

In [ ]:
chunk_embeddings[:2]

array([[-1.1006908 , -0.37803537, -0.39443707, ...,  0.14159042,
        -0.37158853, -0.21305898],
       [-0.49802774,  0.06850217, -0.2888093 , ..., -0.09489366,
         0.12228046, -0.23319054]], dtype=float32)

In [ ]:
# 배열 저장
np.save('/content/drive/MyDrive/nlp-project/chunk_embeddings.npy', chunk_embeddings)

In [ ]:
import gdown, os, glob, shutil
import numpy as np
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

# 파일 다운로드
file_id = '1212HCwiIB2aVBr-e4afjaXcv56npdUhw'
download_file(file_id, 'chunk_embeddings.npy')

chunk_embeddings = np.load('/content/chunk_embeddings.npy')

Downloading...
From (original): https://drive.google.com/uc?id=1212HCwiIB2aVBr-e4afjaXcv56npdUhw
From (redirected): https://drive.google.com/uc?id=1212HCwiIB2aVBr-e4afjaXcv56npdUhw&confirm=t&uuid=31414ad8-194b-4069-815e-6bed53c8d7ed
To: /content/chunk_embeddings.npy
100%|██████████| 3.06G/3.06G [00:15<00:00, 192MB/s]


### 7.2. SentenceTransformers 검색 함수
- SentenceTransformers를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model_st.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    return rank[:top_n]

In [ ]:
query = '과학에 대하여 알려줘'
query_sentence_transformer(model_st, chunk_embeddings, query)

array([466576, 584236, 555455, 941465, 493317, 745323, 748368,  43467,
       166789, 175165])

### 7.3. SLLM RAG with SentenceTransformers를 구현
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문에 대해서 관련된 문서를 SentenceTransformers를를 이용해서 5개 구합니다.
  - 질문과 함께 문서 5개를 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

In [ ]:
def SentenceTransformers_query(model, chunk_embeddings, query, chunk_list):

    result = query_sentence_transformer(model, chunk_embeddings, query)
    print(result)
    info_list = [chunk_list[i] for i in result]
    print(info_list)
    messages = [
    {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해라.:

        문서5: {info_list[4]}

        문서4: {info_list[3]}

        문서3: {info_list[2]}

        문서2: {info_list[1]}

        문서1: {info_list[0]}

        질문: {query}"""
    }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)
    # print(prompt)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    print('답변 \n', outputs[0]["generated_text"][len(prompt):])

In [ ]:
query = '과학이 뭐야?'
SentenceTransformers_query(model_st, chunk_embeddings, query, chunk_list)

[466576 584236 555455 513998 748368 175165 745323   6554 341161  28176]
['과학로\n과학로는 다음을 가리키는 말이다.', '과학원\n과학원의 다른 뜻은 다음과 같다.', '과학 산업 박물관\n과학 산업 박물관은 다음과 같은 박물관이 있다.', '부트스트랩 모형\n부트스트랩 모형()은 양자 색역학 이전에 통용되던 물리 모형으로, 일반론으로부터 복잡한 이론을 완성하고자 하는 연구 분야이다.', '운동 법칙\n물리학에서 여러 운동 법칙이 이론이 개발되었다. 그 중 다음을 포함한다:', '우주과학\n우주과학()은 우주에 대해 연구하는 과학을 한데 묶어 부르는 이름이다.\n연구 현상.\n이외에도 우주에서 일어나는 여러 현상들을 연구한다.', '이공주\n이공주는 다음 사람을 가리킨다.', '다윈\n다윈의 다른 뜻은 다음과 같다.', '리가 1\n리가 1의 다른 뜻은 다음과 같다.', '이론물리학\n1900년대 중반을 지나는 때의 이론 물리 경향은 이전까지 물리학에서 쓰지 않던 수학들이 현대 물리의 전반에 응용이 되어 널리 쓰이게 되었다는 점이다. 리 군과 리 대수는 게이지 대칭성에 쓰이며, 일반 상대성 이론, 양자장론에서 장을 기술하는데 준 리만 다양체를 사용한다. 그러나 초끈 이론은 2021년까지 검증된것이 하나도 없어서 많은 비판을 받고 있으며, 물리학이라기보다는 아직은 양자장론을 도와주는 방법론 정도에 그치고 있다. 이론 물리학이라기보다는 수리 물리학에 가깝다고 해야겠다. 현대 물리학에서 실험과 거리가 있으면서 지나치게 수학에 의존이 심한 이론들이 나오는 경향은 물리학자들의 경계를 받기도 하며, 특히 과학을 하는데 수학적 아름다움 비슷한걸 고려하는 것 같은 비과학적인 방식을 지나치게 중요시하는 물리학자들이 비판을 받기도 한다.\n대표적인 이론물리학자로는 알베르트 아인슈타인, 막스 플랑크, 에르빈 슈뢰딩거, 하이젠베르크, 리처드 파인만, 로버트 오펜하이머, 스티븐 와인버그, 폴 디랙, 스티븐 호킹,킵 손 등이 

## 8. RAG 성능 확장 실습 (재시작 필요)
- CHUNK_FN의 전체 문서를 원하는 데이터로 구성해 보세요.
  - 예) 다른 위키 문서들
  - 예) 뉴스
  - 예) 책 또는 메뉴얼
- 프롬프트를 개선해 보세요.
- 'google/gemma-1.1-7b-it' 까지는 T4에서 겨우 돌아갑니다. 큰 LLM을 적용해 보세요.
- 다양한 방법으로 자신만의 RAG QA봇을 만들어 보세요.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.


- 입력하려는 문서의 수를 늘리니 메모리 부족문제가 발생하여 2B 사용하여
- bm25와 SentenceTransformers를 통해 얻은 유사한 정보를 각 10개씩 입력하여 RAG 실행

In [ ]:
import gdown, os, glob, shutil
import csv

def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

# 파일 다운로드
file_id = '1wrlIHXvZXLBoiPjH2nIPURdceMusQdrj'
download_file(file_id, 'chunk_list.csv')
file_id = '1fBk2ohMp1GJbGrSi8P0-zcIugW-csHUi'
download_file(file_id, "tokenized_chunks.csv")


#  문서를 이미 chunk 단위로 분할한 chunk_list 파일에서 불러오기
with open('/content/chunk_list.csv', "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    chunk_list = next(reader)
# chunk_list를 tokenize한 tokenized_chunks 불러오기
tokenized_chunks = []
with open('/content/tokenized_chunks.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        tokenized_chunks.append(row)
# chunk_embeddings
file_id = '1212HCwiIB2aVBr-e4afjaXcv56npdUhw'
download_file(file_id, 'chunk_embeddings.npy')

chunk_embeddings = np.load('/content/chunk_embeddings.npy')

chunk_list.csv 파일이 이미 존재합니다.
tokenized_chunks.csv 파일이 이미 존재합니다.
chunk_embeddings.npy 파일이 이미 존재합니다.


In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer1(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [ ]:
# bm25 api 생성
bm25 = BM25Okapi(tokenized_chunks)

def query_bm25(bm25, query, tokens, top_n=10):
    doc_scores = bm25.get_scores(tokens)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    return rank[:top_n]

In [ ]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model_st.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    return rank[:top_n]

In [ ]:
# SentenceTransformer 모델 생성
model_st = SentenceTransformer(SEARCH_MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

#pipe line
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
def rag_query(bm25, model, chunk_embeddings, query, tokens, chunk_list):
    result1 = query_bm25(bm25, query, tokens)
    # print(result1)
    info_list1 = [chunk_list[i] for i in result1]

    result2 = query_sentence_transformer(model, chunk_embeddings, query)
    # print(result2)
    info_list2 = [chunk_list[i] for i in result2]
    messages = [
    {
        "role": "user",
        "content": f"""'문서1'부터 '문서15'를 참고하고 너의 지식을 이용하여 '질문'의 문맥에 맞게 답변하라.:


        문서15: {info_list2[9]}

        문서14: {info_list2[8]}

        문서13: {info_list2[7]}

        문서12: {info_list2[6]}

        문서11: {info_list2[5]}

        문서10: {info_list1[4]}

        문서9: {info_list1[3]}

        문서8: {info_list1[2]}

        문서7: {info_list1[1]}

        문서6: {info_list1[0]}

        문서5: {info_list2[4]}

        문서4: {info_list2[3]}

        문서3: {info_list2[2]}

        문서2: {info_list2[1]}

        문서1: {info_list2[0]}

        질문: {query}"""
    }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)
    # print(prompt)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    print('답변 \n', outputs[0]["generated_text"][len(prompt):])

In [ ]:
while True:
    query = input('질문 : ')
    if len(query) == 0:
        break
    tokens = tokenizer1(query)
    rag_query(bm25, model_st, chunk_embeddings, query, tokens, chunk_list)
    print('-'*100)

질문 : 손흥민이 누구야?
답변 
 문서에서 제공된 정보를 통해 손흥민의 이름은 언제나 나오지 않아 답변할 수 없습니다.
----------------------------------------------------------------------------------------------------
질문 : 저현고등학교가 뭐야?
답변 
 문서3: 저현고등학교는 경기도 고양시 일산동구 식사동에 있는 공립 고등학교이다.
----------------------------------------------------------------------------------------------------
질문 : 아이폰의 역사
답변 
 **아이폰의 역사:**

- 2007년: iPhone 2G- 1세대 아이폰이 출시, 멀티터치 기능과 2세대 통신망 기술 지원을 제공했다.
- 2008년: iPhone 3G- 2세대 아이폰이 출시, 3세대 통신망 기술 지원을 제공했다.
- 2009년: iPhone 3GS- 3세대 아이폰이 출시, 동영상 촬영이 가능하게 되었다.
- 2010년: iPhone 4- 4세대 아이폰이 출시, 디자인적 변화 요소가 많았다.
- 2011년: iPhone 4s- 5세대 아이폰이 출시, 스티브 잡스의 유작이라 불린다.
- 2012년: iPhone 5- 6세대 아이폰이 출시, 4인치 디스플레이와 4G 지원을 제공했다.
- 2014년: 아이폰 13이 출시, 6.1인치 디스플레이와 2015년 11월 11일 출시되었다.
- 2019년: 아이폰 11 프로가 출시, 아이폰 11과 같은 기능을 제공했다.
- 2021년: 아이폰 13 프로와 아이폰 13 프로 맥스가 출시, 16세대 플래그십 아이폰으로 출시되었다.
----------------------------------------------------------------------------------------------------
질문 : 삼성 갤럭시의 역사
답변 
 **질문: 삼성 갤럭시의 역사**

문서를

## 9. 향후 성능 향상을 위한 방법에 대한 제안
- 문서 분할을 기계적 분할이 아닌 GPT-4를 이용해서 의미 기반으로 분할하는 방법도 있습니다.
- 검색 성능 향상을 위해서는 DPR 계열의 기술을 사용하길 추천합니다.
- 검색 성능 향상을 위해서 BM25와 DPR을 적절한 비율로 반영하면 더 좋은 성능을 얻을 수 있다는 연구 결과가 있습니다.
- 많은 문서를 사용하기 위해서 벡터DB를 사용하는 것을 추천드립니다.
- RAG 성능을 높이는 방법으로는 추가적인 fine-tuining을 해 보는 것을 추천드립니다.
- RAG를 학습을 위한 데이터는 예상되는 질문 및 관련 문서 쌍을 1,000개 이상 만들고 GPT-4에게 질문을 해보고 그 답변을 정답으로 사용해서, SLLM을 fine-tuining 해 보는 것도 좋은 방법입니다.